<a href="https://colab.research.google.com/github/ReenanNunes/compiladores-2024-t1/blob/main/Mapa_de_Risco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install contextily
!git clone https://github.com/ReenanNunes/Mapa-de-Risco



import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

def calcular_menor_distancia(origem, destinos):
    destino_mais_proximo = destinos.geometry.apply(lambda x: origem.distance(x)).idxmin()
    return destinos.loc[destino_mais_proximo]

chuva_df = pd.read_csv("/content/dados/dados_chuva.csv")
elevacao_df = pd.read_csv("/content/dados/dados_elevacao.csv")
densidade_df = pd.read_csv("/content/dados/densidade_populacional.csv")
inundacoes_df = pd.read_csv("/content/dados/inundacoes_historicas.csv")

chuva_gdf = gpd.GeoDataFrame(
    chuva_df,
    geometry=gpd.points_from_xy(chuva_df.longitude, chuva_df.latitude),
    crs="EPSG:4326"
)
elevacao_gdf = gpd.GeoDataFrame(
    elevacao_df,
    geometry=gpd.points_from_xy(elevacao_df.longitude, elevacao_df.latitude),
    crs="EPSG:4326"
)
densidade_gdf = gpd.GeoDataFrame(
    densidade_df,
    geometry=gpd.points_from_xy(densidade_df.longitude, densidade_df.latitude),
    crs="EPSG:4326"
)
inundacoes_gdf = gpd.GeoDataFrame(
    inundacoes_df,
    geometry=gpd.points_from_xy(inundacoes_df.longitude, inundacoes_df.latitude),
    crs="EPSG:4326"
)

chuva_gdf = chuva_gdf.to_crs("EPSG:3857")
elevacao_gdf = elevacao_gdf.to_crs("EPSG:3857")
densidade_gdf = densidade_gdf.to_crs("EPSG:3857")
inundacoes_gdf = inundacoes_gdf.to_crs("EPSG:3857")

chuva_gdf["elevacao_m"] = chuva_gdf.apply(lambda row: calcular_menor_distancia(row.geometry, elevacao_gdf)["elevacao_m"], axis=1)
chuva_gdf["densidade_populacional"] = chuva_gdf.apply(lambda row: calcular_menor_distancia(row.geometry, densidade_gdf)["densidade_populacional"], axis=1)

FATOR_CHUVA = 0.5
FATOR_ELEVACAO = 0.3
FATOR_DENSIDADE = 0.2

chuva_gdf["risco_inundacao"] = (
    chuva_gdf["chuva_mm"] * FATOR_CHUVA +
    (1 / chuva_gdf["elevacao_m"]) * FATOR_ELEVACAO +
    chuva_gdf["densidade_populacional"] * FATOR_DENSIDADE
)

if chuva_gdf.crs.to_string() != "EPSG:3857":
    chuva_gdf = chuva_gdf.to_crs("EPSG:3857")
if inundacoes_gdf.crs.to_string() != "EPSG:3857":
    inundacoes_gdf = inundacoes_gdf.to_crs("EPSG:3857")

if "categoria_risco" not in chuva_gdf.columns:
    print("Criando a coluna 'categoria_risco'...")
    chuva_gdf["categoria_risco"] = pd.cut(
        chuva_gdf["risco_inundacao"],
        bins = [0, 2, 4, float('inf')],
        labels=["Baixo", "Médio", "Alto"],
        include_lowest=True
    )

if chuva_gdf.empty:
    raise ValueError("O GeoDataFrame de chuva está vazio. Verifique os dados de entrada e processamento.")

print("Colunas disponíveis:", chuva_gdf.columns)
print("Valores únicos em 'categoria_risco':", chuva_gdf["categoria_risco"].unique())

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
base = chuva_gdf.plot(
    column="categoria_risco",
    cmap="Reds",
    linewidth=0.8,
    ax=ax,
    edgecolor="0.8",
    legend=True
)


ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, crs=chuva_gdf.crs)

plt.title("Mapa Detalhado de Risco de Inundações", fontsize=16)
plt.legend(loc="upper right", fontsize=10)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.tight_layout()
plt.show()

Cloning into 'Mapa-de-Risco'...
fatal: could not read Username for 'https://github.com': No such device or address
